<a href="https://colab.research.google.com/github/UnpackJungHo/XRSimulator_Osaka/blob/Learning_AI/LSTM_TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import glob
import os

# scikit-learn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


# ========== 1. 여러 파일(2014~2023) 불러와서 전처리 후 병합 ==========
def load_and_preprocess_data_multiple(folder_path='.', start_year=2014, end_year=2023):
    """
    2014.weather.xlsx ~ 2023.weather.xlsx 파일을 모두 읽어 하나의 DataFrame으로 결합.
    folder_path: 파일들이 위치한 폴더 경로
    start_year, end_year: 합칠 XLSX 파일 범위
    """
    all_dfs = []
    for year in range(start_year, end_year + 1):
        file_name = f"{year}_weather.xlsx"
        file_path = os.path.join(folder_path, file_name)
        if os.path.exists(file_path):
            df_temp = pd.read_excel(file_path)

            # DateTime 컬럼 시계열 변환
            df_temp['DateTime'] = pd.to_datetime(df_temp['DateTime(YYYYMMDDHHMI)'], format='%Y%m%d%H%M')
            df_temp.set_index('DateTime', inplace=True)

            # -9를 결측치로 처리 후 보간(interpolate) - 필요에 따라 변경 가능
            df_temp.replace(-9, np.nan, inplace=True)
            df_temp.interpolate(method='time', inplace=True)

            # 필요 없는 컬럼(STN 등) 제거 (있다면)
            if 'STN' in df_temp.columns:
                df_temp.drop(columns=['STN'], inplace=True)

            all_dfs.append(df_temp)
        else:
            print(f"File not found: {file_path}")

    # 여러 해 데이터를 하나로 합침
    if len(all_dfs) == 0:
        raise ValueError("No data files found in the specified range.")

    df_merged = pd.concat(all_dfs)
    df_merged.sort_index(inplace=True)  # 시간 순으로 정렬
    return df_merged


def load_and_preprocess_single(file_path='2024_weather.xlsx'):
    """
    2024.weather.xlsx 파일을 읽어온 뒤 전처리 (테스트용)
    """
    df = pd.read_excel(file_path)
    df['DateTime'] = pd.to_datetime(df['DateTime'], format='%Y%m%d%H%M')
    df.set_index('DateTime', inplace=True)

    df.replace(-9, np.nan, inplace=True)
    df.interpolate(method='time', inplace=True)

    if 'STN' in df.columns:
        df.drop(columns=['STN'], inplace=True)

    df.sort_index(inplace=True)
    return df


# ========== 2. 랜덤 포레스트 학습 (예: TA_next1 예측) ==========
def train_random_forest(df):
    """
    랜덤 포레스트 모델을 학습한다.
    예시로, 현재 시점의 TA → 1시간 뒤 TA를 예측하도록 구성한다.
    """
    # 1시간 뒤 온도를 타깃으로 하는 컬럼 생성
    df['TA_next1'] = df['TA'].shift(-1)

    # 결측치 제거
    df_rf = df.dropna(subset=['TA', 'TA_next1'])

    # 예시로 사용할 특징
    features = ['TA', 'RN', 'SD_TOT', 'CA_TOT', 'WD', 'WS', 'HM']
    # 실제 데이터에 맞춰 필요 없는 것은 제외, 필요한 것은 추가

    # 모델 입력/타깃
    X = df_rf[features]
    y = df_rf['TA_next1']

    # 시계열 특성상 shuffle=False 권장
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, shuffle=False
    )

    # 랜덤 포레스트 모델 학습
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)

    # 검증 (필요하면 mse, rmse, r2 등 계산 가능)
    y_pred_rf = rf_model.predict(X_val)
    mse_rf = mean_squared_error(y_val, y_pred_rf)
    print("[RandomForest] Validation MSE:", mse_rf)

    return rf_model


# ========== 3. LSTM 학습을 위한 시계열 데이터셋 구성 ==========
def create_lstm_dataset(dataset, target, seq_length=24):
    """
    시계열 데이터셋을 (samples, seq_length, features) 형태로 변환.
    dataset: 입력 피처들을 담은 DataFrame
    target: 예측하고자 하는 시리즈 (ex. df['TA'])
    seq_length: 몇 시간의 과거 기록을 볼지
    """
    Xs, ys = [], []
    for i in range(len(dataset) - seq_length):
        Xs.append(dataset.iloc[i:i+seq_length].values)
        ys.append(target.iloc[i+seq_length])
    return np.array(Xs), np.array(ys)


# ========== 4. LSTM 모델 학습 ==========
def train_lstm(df, seq_length=24):
    """
    LSTM 모델을 학습하고, 학습 완료된 모델을 반환한다.
    df: 전처리된 데이터 (시간 인덱스 설정됨)
    seq_length: LSTM이 볼 과거 시간 길이
    """
    # LSTM에서 사용할 피처 예시
    lstm_features = ['TA', 'WS', 'HM']  # 프로젝트 맞게 변경 가능
    df_lstm = df.dropna(subset=lstm_features)

    # X_lstm, y_lstm 구성 (ex. 'TA' 예측)
    X_lstm, y_lstm = create_lstm_dataset(
        df_lstm[lstm_features],
        df_lstm['TA'],
        seq_length
    )

    # 훈련/검증 분할
    split_idx = int(len(X_lstm) * 0.8)
    X_train, X_val = X_lstm[:split_idx], X_lstm[split_idx:]
    y_train, y_val = y_lstm[:split_idx], y_lstm[split_idx:]

    # 모델 설계
    model = Sequential()
    model.add(LSTM(64, input_shape=(seq_length, len(lstm_features)), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(32, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mse')

    # 조기 종료
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # 학습
    model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_val, y_val),
        callbacks=[early_stop],
        verbose=1
    )

    # 검증 MSE
    y_pred_val = model.predict(X_val)
    mse_lstm = mean_squared_error(y_val, y_pred_val)
    print("[LSTM] Validation MSE:", mse_lstm)

    return model


# ========== 5. LSTM 예측 함수 (특정 날짜로 24시간 단위 예측) ==========
def predict_for_date(df, model, target_date_str='2025/02/23', seq_length=24):
    """
    특정 날짜(예: 2025/02/23)에 대해 24시간(00~23시) 예측.
    df: 전처리된 원본 데이터프레임
    model: 학습된 LSTM 모델
    target_date_str: 예측할 날짜 (YYYY/MM/DD)
    seq_length: LSTM이 볼 과거 시간 길이
    """
    target_date = pd.to_datetime(target_date_str)

    # LSTM에서 사용한 피처 (train_lstm 함수와 동일하게 맞춰야 함)
    lstm_features = ['TA', 'WS', 'HM']

    # 최근 24시간치 (seq_length만큼) 데이터를 가져온다.
    start_time = target_date - pd.Timedelta(hours=seq_length)

    # 필요 범위의 데이터가 충분히 존재해야 하므로 인덱스 확인 필수
    recent_data = df.loc[start_time:target_date].copy()

    # 혹시 누락된 시간이 있으면 보간 등을 통해 처리
    recent_data.interpolate(method='time', inplace=True)

    # 입력 시퀀스 만들기
    input_seq = recent_data[lstm_features].values[-seq_length:]  # (seq_length, len(lstm_features))

    # 24시간 예측 반복
    prediction_result = []
    for hour in range(24):
        # (1, seq_length, feature)로 reshape
        X_input = np.array([input_seq])

        # 1시간 뒤 TA 예측
        ta_pred = model.predict(X_input)[0, 0]

        # 예측 시간
        pred_time = target_date + pd.Timedelta(hours=hour)

        # 결과 저장 (여기서는 RN, SD_TOT 등은 -9 또는 임의값으로 둔 예시)
        prediction_result.append([
            pred_time.strftime('%Y%m%d%H%M'),  # 예: '202502230000'
            108,   # STN (가정)
            ta_pred,
            -9,    # RN
            -9,    # SD_TOT
            -9,    # CA_TOT
            -9,    # WD
            -9,    # WS
            -9,    # HM
        ])

        # 다음 시점을 위해 input_seq 업데이트
        new_row = input_seq[-1].copy()
        new_row[0] = ta_pred  # TA 업데이트
        input_seq = np.vstack([input_seq[1:], new_row])

    # 결과를 DataFrame으로 변환
    columns = [
        'DateTime(YYYYMMDDHHMI)',
        'STN',
        'TA',
        'RN',
        'SD_TOT',
        'CA_TOT',
        'WD',
        'WS',
        'HM'
    ]
    result_df = pd.DataFrame(prediction_result, columns=columns)
    return result_df


# ========== 6. 테스트(2024) 데이터에 대한 예측 및 실제 비교 ==========
def evaluate_on_test_data(df_test, model, seq_length=24):
    """
    2024년 전체 데이터를 대상으로 LSTM 예측값과 실제값을 비교하여
    MSE를 구하는 예시 함수.
    (예시: TA를 대상으로 함)
    """
    # LSTM과 동일한 피처
    lstm_features = ['TA', 'WS', 'HM']
    df_test_lstm = df_test.dropna(subset=lstm_features).copy()

    # 시계열 형태로 변환
    X_test, y_test = create_lstm_dataset(
        df_test_lstm[lstm_features],
        df_test_lstm['TA'],
        seq_length
    )

    if len(X_test) == 0:
        raise ValueError("Not enough data in test set for the given seq_length.")

    # 예측
    y_pred = model.predict(X_test)

    # MSE 계산
    mse_test = mean_squared_error(y_test, y_pred)
    print("[LSTM] Test MSE on 2024:", mse_test)

    # 필요하면 예측값과 실제값을 DataFrame으로 확인
    # 인덱스 맞추기
    y_test_index = df_test_lstm.index[seq_length:]
    compare_df = pd.DataFrame({
        'Actual_TA': y_test,
        'Predicted_TA': y_pred.flatten()
    }, index=y_test_index)

    return compare_df, mse_test


# ========== 7. 전체 실행 흐름 예시 ==========
if __name__ == '__main__':
    # 1) 2014~2023년 파일을 모두 불러와서 학습 데이터프레임 생성
    train_df = load_and_preprocess_data_multiple(
        folder_path='.',  # 파일 위치한 폴더
        start_year=2014,
        end_year=2023
    )

    # 2) 2024년 데이터 불러오기(테스트용)
    test_df_2024 = load_and_preprocess_single('2024_weather.xlsx')

    # 3) 랜덤 포레스트 모델 학습 (선택적으로 사용)
    rf_model = train_random_forest(train_df)

    # 4) LSTM 모델 학습
    lstm_model = train_lstm(train_df, seq_length=24)

    # 5) 2024년 예측 vs 실제 비교 (LSTM 기준)
    compare_df_2024, mse_2024 = evaluate_on_test_data(test_df_2024, lstm_model, seq_length=24)
    print(compare_df_2024.head(10))  # 일부 결과 미리보기

    # 6) (옵션) 특정 날짜에 대한 24시간 예측 결과 출력
    pred_sample = predict_for_date(test_df_2024, lstm_model, '2024/01/05', seq_length=24)
    print(pred_sample)


<ipython-input-4-76bc584d3e61>:38: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_temp.interpolate(method='time', inplace=True)
<ipython-input-4-76bc584d3e61>:38: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_temp.interpolate(method='time', inplace=True)
<ipython-input-4-76bc584d3e61>:38: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_temp.interpolate(method='time', inplace=True)
<ipython-input-4-76bc584d3e61>:38: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_temp.interpolate(metho

KeyError: 'DateTime'